In [1]:
from tqdm.notebook import tqdm
import torch
from torch import nn
import torch.optim as optim
import torch.nn.functional as F
import torch.autograd.profiler as profiler

import matplotlib.pyplot as plt

import torchvision
import torchvision.transforms as transforms


GRAPH_PATH = "./data/graphs/exp_conv_pad.jpg"
DATA_DIR = './data/MNIST'

In [2]:
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_VISIBLE_DEVICES=0


In [3]:
def load_data(data_dir, batch_size):
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])
    
    trainset = torchvision.datasets.MNIST(
        root=data_dir, train=True, download=True, transform=transform)

    testset = torchvision.datasets.MNIST(
        root=data_dir, train=False, download=True, transform=transform)
    
    train_loader = torch.utils.data.DataLoader(trainset, batch_size)
    test_loader = torch.utils.data.DataLoader(testset, batch_size)

    return train_loader, test_loader

In [4]:
in_channels = 3
out_channels = 10
device = "cuda:0"

train_loss = nn.CrossEntropyLoss()

learning_rate = 1e-4

batch_size = 256
epoch_num = 5

In [5]:
trainset, testset = load_data(data_dir=DATA_DIR, batch_size=batch_size)

In [6]:
def train(net, optimizer, criterion, epoch_num, train_data, device, graph_path):
    """Neural network training process.
    
    """
    for epoch in tqdm(range(epoch_num)):  # loop over the dataset multiple times
        for data, target in train_data:
            inputs, labels = data.to(device), target.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()    
    return net

In [7]:
class Network(nn.Module):
  def __init__(self):
    super().__init__()
    self.convs = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=6, kernel_size=3, padding=1),
            nn.BatchNorm2d(6),
            nn.ReLU(),
        
            nn.Conv2d(in_channels=6, out_channels=12, kernel_size=3, padding=1),
            nn.BatchNorm2d(12),
            nn.ReLU(),
        
            nn.Conv2d(in_channels=12, out_channels=24, kernel_size=3, padding=1),
            nn.BatchNorm2d(24),
            nn.ReLU()        
        )

    self.out = nn.Linear(in_features=24, out_features=10)

  def forward(self, tensor):
    tensor = self.convs(tensor)   

    # output
    tensor = nn.AdaptiveAvgPool2d((1,1))(tensor) 
    tensor = torch.flatten(tensor, 1)
    tensor = self.out(tensor)
    tensor = torch.sigmoid(tensor)

    return tensor

In [8]:
net = Network()
net.to(device)

optimizer = optim.Adam(net.parameters(), lr=learning_rate)

In [ ]:
with torch.autograd.profiler.profile(use_cuda=True) as prof_train:
    net = train(
                net, optimizer, train_loss, epoch_num, trainset, device, GRAPH_PATH
    )

  0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
prof.table()

In [13]:
# import torch
# import torchvision.models as models

# model = models.densenet121(pretrained=True)
# x = torch.randn((1, 3, 224, 224), requires_grad=True)

with torch.autograd.profiler.profile(use_cuda=True) as prof_inference:
    net(testset)

In [18]:
with open("data/profile_info/prof_conv_with_pad.txt", "w") as file:
    file.write(prof.table())

In [ ]:
# https://docs.nvidia.com/deeplearning/frameworks/pyprof-user-guide/profile.html 
# https://pytorch.org/tutorials/beginner/profiler.html